# atmodeller

## Tutorial 1: basic operation

Import the required packages and set the package logger to the INFO level. For more output you could instead set it to DEBUG.

In [1]:
from atmodeller import InteriorAtmosphereSystem, Molecule, SystemConstraint, OCEAN_MOLES, MolarMasses, logger
from atmodeller.thermodynamics import PeridotiteH2O, NoSolubility, BasaltDixonCO2
import logging

logger.setLevel(logging.INFO)

[2023-06-04 14:05:57 - atmodeller           - 023 - INFO      - <module>()] atmodeller version 0.1.0


### 1. Simple H2O-H2 system with prescribed H2O pressure

We define a list of the molecules we wish to include in the interior-atmosphere system and how they partition between the melt and the atmosphere, and the solid and the melt:

In [2]:
molecules: list[Molecule] = []
molecules.append(Molecule(name='H2O', solubility=PeridotiteH2O(), solid_melt_distribution_coefficient=0))
molecules.append(Molecule(name='H2', solubility=NoSolubility(), solid_melt_distribution_coefficient=0))

[2023-06-04 14:05:58 - atmodeller.core      - 144 - INFO      - __post_init__()] Creating a molecule: H2O
[2023-06-04 14:05:58 - atmodeller.core      - 144 - INFO      - __post_init__()] Creating a molecule: H2


Although a choice is often made to constrain oxygen fugacity, we nevertheless need to explicitly include O2 as a molecule in the interior-atmosphere system:

In [3]:
molecules.append(Molecule(name='O2', solubility=NoSolubility(), solid_melt_distribution_coefficient=0))
molecules

[2023-06-04 14:05:58 - atmodeller.core      - 144 - INFO      - __post_init__()] Creating a molecule: O2


[Molecule(name='H2O', solubility=<atmodeller.thermodynamics.PeridotiteH2O object at 0x11ae3cd90>, solid_melt_distribution_coefficient=0, elements={'H': 2, 'O': 1}, element_masses={'H': 0.0020158, 'O': 0.0159994}, formation_constants=(-3.0385132352941198, 13152.698529411768), molar_mass=0.018015200000000002),
 Molecule(name='H2', solubility=<atmodeller.thermodynamics.NoSolubility object at 0x116d6a690>, solid_melt_distribution_coefficient=0, elements={'H': 2}, element_masses={'H': 0.0020158}, formation_constants=(0, 0), molar_mass=0.0020158),
 Molecule(name='O2', solubility=<atmodeller.thermodynamics.NoSolubility object at 0x127828b10>, solid_melt_distribution_coefficient=0, elements={'O': 2}, element_masses={'O': 0.0319988}, formation_constants=(0, 0), molar_mass=0.0319988)]

We can then create an interior-atmosphere system using the list of molecules. Note that this creates a planet with 'default properties' (a reduced and molten Earth). Adjusting the planet properties will be covered in a later tutorial.

In [4]:
interior_atmosphere: InteriorAtmosphereSystem = InteriorAtmosphereSystem(molecules=molecules)

[2023-06-04 14:05:58 - atmodeller.core      - 072 - INFO      - __post_init__()] Creating a new planet
[2023-06-04 14:05:58 - atmodeller.core      - 077 - INFO      - __post_init__()] Mantle mass (kg) = 4.208261222595111e+24
[2023-06-04 14:05:58 - atmodeller.core      - 078 - INFO      - __post_init__()] Mantle melt fraction = 1.0
[2023-06-04 14:05:58 - atmodeller.core      - 079 - INFO      - __post_init__()] Core mass fraction = 0.295334691460966
[2023-06-04 14:05:58 - atmodeller.core      - 080 - INFO      - __post_init__()] Planetary radius (m) = 6371000.0
[2023-06-04 14:05:58 - atmodeller.core      - 081 - INFO      - __post_init__()] Planetary mass (kg) = 5.972e+24
[2023-06-04 14:05:58 - atmodeller.core      - 082 - INFO      - __post_init__()] Surface temperature (K) = 2000.000000
[2023-06-04 14:05:58 - atmodeller.core      - 083 - INFO      - __post_init__()] Surface gravity (m/s^2) = 9.819973426224687
[2023-06-04 14:05:58 - atmodeller.core      - 084 - INFO      - __post_init_

Notice that for this simple system it has identified the single reaction that relates the 3 chosen molecules. To solve the system, we provide a constraint of the H2O pressure in bar:

In [5]:
H2O_pressure: SystemConstraint = SystemConstraint(species='H2O', value=1, field='pressure')
constraints: list[SystemConstraint] = [H2O_pressure]

A second constraint is required to close the system of equations, and this is conveniently specified using `fo2_constraint=True`, which (without specifying additional arguments) will impose the oxygen fugacity at the Iron-Wustite buffer. Hence we can solve the interior-atmosphere system at a given temperature to determine the partial pressures of our chosen molecules:

In [6]:
interior_atmosphere.solve(constraints, fo2_constraint=True)

[2023-06-04 14:05:58 - atmodeller.core      - 756 - INFO      - solve()] Constraints: [SystemConstraint(species='H2O', value=1, field='pressure')]
[2023-06-04 14:05:58 - atmodeller.core      - 766 - INFO      - solve()] Pressure constraints only so attempting to solve a linear reaction network
[2023-06-04 14:05:58 - atmodeller.core      - 637 - INFO      - solve()] Solving the reaction network
[2023-06-04 14:05:58 - atmodeller.core      - 557 - INFO      - get_coefficient_matrix_and_rhs()] Adding fO2 as an additional constraint using IronWustiteBufferOneill with fO2_shift = 0.00
[2023-06-04 14:05:58 - atmodeller.core      - 576 - INFO      - get_coefficient_matrix_and_rhs()] The necessary number of constraints will be applied to the reaction network to solve the system
[2023-06-04 14:05:58 - atmodeller.core      - 591 - INFO      - get_coefficient_matrix_and_rhs()] Row 00: Reaction 0: 1.0 H2 + 0.5 O2 = 1.0 H2O
[2023-06-04 14:05:58 - atmodeller.core      - 605 - INFO      - get_coeffici

{'H2': 1.465803963408939, 'O2': 3.9099946023954954e-08, 'H2O': 1.0}

You can access the solution directly using:

In [7]:
interior_atmosphere.pressures_dict

{'H2': 1.465803963408939, 'O2': 3.9099946023954954e-08, 'H2O': 1.0}

### 2. System with C and H and prescribed pressures

We now extend the molecule list to additionally include C-species:

In [8]:
molecules: list[Molecule] = []
molecules.append(Molecule(name='H2O', solubility=PeridotiteH2O(), solid_melt_distribution_coefficient=0))
molecules.append(Molecule(name='H2', solubility=NoSolubility(), solid_melt_distribution_coefficient=0))
molecules.append(Molecule(name='O2', solubility=NoSolubility(), solid_melt_distribution_coefficient=0))
molecules.append(Molecule(name='CO', solubility=NoSolubility(), solid_melt_distribution_coefficient=0))
molecules.append(Molecule(name='CO2', solubility=BasaltDixonCO2(), solid_melt_distribution_coefficient=0))
molecules

[2023-06-04 14:05:59 - atmodeller.core      - 144 - INFO      - __post_init__()] Creating a molecule: H2O
[2023-06-04 14:05:59 - atmodeller.core      - 144 - INFO      - __post_init__()] Creating a molecule: H2
[2023-06-04 14:05:59 - atmodeller.core      - 144 - INFO      - __post_init__()] Creating a molecule: O2
[2023-06-04 14:05:59 - atmodeller.core      - 144 - INFO      - __post_init__()] Creating a molecule: CO
[2023-06-04 14:05:59 - atmodeller.core      - 144 - INFO      - __post_init__()] Creating a molecule: CO2


[Molecule(name='H2O', solubility=<atmodeller.thermodynamics.PeridotiteH2O object at 0x12783d0d0>, solid_melt_distribution_coefficient=0, elements={'H': 2, 'O': 1}, element_masses={'H': 0.0020158, 'O': 0.0159994}, formation_constants=(-3.0385132352941198, 13152.698529411768), molar_mass=0.018015200000000002),
 Molecule(name='H2', solubility=<atmodeller.thermodynamics.NoSolubility object at 0x11a550bd0>, solid_melt_distribution_coefficient=0, elements={'H': 2}, element_masses={'H': 0.0020158}, formation_constants=(0, 0), molar_mass=0.0020158),
 Molecule(name='O2', solubility=<atmodeller.thermodynamics.NoSolubility object at 0x11ae33050>, solid_melt_distribution_coefficient=0, elements={'O': 2}, element_masses={'O': 0.0319988}, formation_constants=(0, 0), molar_mass=0.0319988),
 Molecule(name='CO', solubility=<atmodeller.thermodynamics.NoSolubility object at 0x116ce06d0>, solid_melt_distribution_coefficient=0, elements={'C': 1, 'O': 1}, element_masses={'C': 0.0120107, 'O': 0.0159994}, for

In [9]:
interior_atmosphere: InteriorAtmosphereSystem = InteriorAtmosphereSystem(molecules=molecules)

[2023-06-04 14:05:59 - atmodeller.core      - 072 - INFO      - __post_init__()] Creating a new planet
[2023-06-04 14:05:59 - atmodeller.core      - 077 - INFO      - __post_init__()] Mantle mass (kg) = 4.208261222595111e+24
[2023-06-04 14:05:59 - atmodeller.core      - 078 - INFO      - __post_init__()] Mantle melt fraction = 1.0
[2023-06-04 14:05:59 - atmodeller.core      - 079 - INFO      - __post_init__()] Core mass fraction = 0.295334691460966
[2023-06-04 14:05:59 - atmodeller.core      - 080 - INFO      - __post_init__()] Planetary radius (m) = 6371000.0
[2023-06-04 14:05:59 - atmodeller.core      - 081 - INFO      - __post_init__()] Planetary mass (kg) = 5.972e+24
[2023-06-04 14:05:59 - atmodeller.core      - 082 - INFO      - __post_init__()] Surface temperature (K) = 2000.000000
[2023-06-04 14:05:59 - atmodeller.core      - 083 - INFO      - __post_init__()] Surface gravity (m/s^2) = 9.819973426224687
[2023-06-04 14:05:59 - atmodeller.core      - 084 - INFO      - __post_init_

Note now the system has identified two reactions in the network. With C present in the system we must provide at least 2 constraints, in addition to the oxygen fugacity:

In [10]:
H2O_pressure: SystemConstraint = SystemConstraint(species='H2O', value=1, field='pressure')
CO2_pressure: SystemConstraint = SystemConstraint(species='CO2', value=1, field='pressure' )
constraints: list[SystemConstraint] = [H2O_pressure, CO2_pressure]

In [11]:
interior_atmosphere.solve(constraints, fo2_constraint=True)
interior_atmosphere.pressures_dict

[2023-06-04 14:05:59 - atmodeller.core      - 756 - INFO      - solve()] Constraints: [SystemConstraint(species='H2O', value=1, field='pressure'),
 SystemConstraint(species='CO2', value=1, field='pressure')]
[2023-06-04 14:05:59 - atmodeller.core      - 766 - INFO      - solve()] Pressure constraints only so attempting to solve a linear reaction network
[2023-06-04 14:05:59 - atmodeller.core      - 637 - INFO      - solve()] Solving the reaction network
[2023-06-04 14:05:59 - atmodeller.core      - 557 - INFO      - get_coefficient_matrix_and_rhs()] Adding fO2 as an additional constraint using IronWustiteBufferOneill with fO2_shift = 0.00
[2023-06-04 14:05:59 - atmodeller.core      - 576 - INFO      - get_coefficient_matrix_and_rhs()] The necessary number of constraints will be applied to the reaction network to solve the system
[2023-06-04 14:05:59 - atmodeller.core      - 591 - INFO      - get_coefficient_matrix_and_rhs()] Row 00: Reaction 0: 1.0 CO + 0.5 O2 = 1.0 CO2
[2023-06-04 14:

{'CO': 6.579409553027913,
 'H2': 1.465803963408939,
 'O2': 3.9099946023954954e-08,
 'CO2': 1.0,
 'H2O': 1.0}

There is not a requirement to necessarily impose the oxygen fugacity as a constraint. Instead, we can simply impose three pressure constraints (that span the reaction set) and allow for the oxygen fugacity to be solved. Note that if we do not specify an appropriate range of constraints we cannot solve the system of equations to give a unique solution and hence the code will raise an exception relating to a singular matrix.

In [12]:
H2O_pressure: SystemConstraint = SystemConstraint(species='H2O', value=1, field='pressure')
H2_pressure: SystemConstraint = SystemConstraint(species='H2', value=1, field='pressure' )
CO_pressure: SystemConstraint = SystemConstraint(species='CO', value=1, field='pressure' )
constraints: list[SystemConstraint] = [H2O_pressure, H2_pressure, CO_pressure]

In [13]:
interior_atmosphere.solve(constraints, fo2_constraint=False)
interior_atmosphere.pressures_dict

[2023-06-04 14:05:59 - atmodeller.core      - 756 - INFO      - solve()] Constraints: [SystemConstraint(species='H2O', value=1, field='pressure'),
 SystemConstraint(species='H2', value=1, field='pressure'),
 SystemConstraint(species='CO', value=1, field='pressure')]
[2023-06-04 14:05:59 - atmodeller.core      - 766 - INFO      - solve()] Pressure constraints only so attempting to solve a linear reaction network
[2023-06-04 14:05:59 - atmodeller.core      - 637 - INFO      - solve()] Solving the reaction network
[2023-06-04 14:05:59 - atmodeller.core      - 576 - INFO      - get_coefficient_matrix_and_rhs()] The necessary number of constraints will be applied to the reaction network to solve the system
[2023-06-04 14:05:59 - atmodeller.core      - 591 - INFO      - get_coefficient_matrix_and_rhs()] Row 00: Reaction 0: 1.0 CO + 0.5 O2 = 1.0 CO2
[2023-06-04 14:05:59 - atmodeller.core      - 591 - INFO      - get_coefficient_matrix_and_rhs()] Row 01: Reaction 1: 1.0 H2 + 0.5 O2 = 1.0 H2O
[

{'CO': 1.0,
 'H2': 1.0,
 'O2': 8.400941126066452e-08,
 'CO2': 0.22278655122394087,
 'H2O': 1.0}

### 3. System with C and H and mixed constraints

A typical use case is to define an interior-atmosphere system with a combination of pressure and mass constraints. We define the same molecule set as before:

In [14]:
molecules: list[Molecule] = []
molecules.append(Molecule(name='H2O', solubility=PeridotiteH2O(), solid_melt_distribution_coefficient=0))
molecules.append(Molecule(name='H2', solubility=NoSolubility(), solid_melt_distribution_coefficient=0))
molecules.append(Molecule(name='O2', solubility=NoSolubility(), solid_melt_distribution_coefficient=0))
molecules.append(Molecule(name='CO', solubility=NoSolubility(), solid_melt_distribution_coefficient=0))
molecules.append(Molecule(name='CO2', solubility=BasaltDixonCO2(), solid_melt_distribution_coefficient=0))
molecules

[2023-06-04 14:06:00 - atmodeller.core      - 144 - INFO      - __post_init__()] Creating a molecule: H2O
[2023-06-04 14:06:00 - atmodeller.core      - 144 - INFO      - __post_init__()] Creating a molecule: H2
[2023-06-04 14:06:00 - atmodeller.core      - 144 - INFO      - __post_init__()] Creating a molecule: O2
[2023-06-04 14:06:00 - atmodeller.core      - 144 - INFO      - __post_init__()] Creating a molecule: CO
[2023-06-04 14:06:00 - atmodeller.core      - 144 - INFO      - __post_init__()] Creating a molecule: CO2


[Molecule(name='H2O', solubility=<atmodeller.thermodynamics.PeridotiteH2O object at 0x127841c50>, solid_melt_distribution_coefficient=0, elements={'H': 2, 'O': 1}, element_masses={'H': 0.0020158, 'O': 0.0159994}, formation_constants=(-3.0385132352941198, 13152.698529411768), molar_mass=0.018015200000000002),
 Molecule(name='H2', solubility=<atmodeller.thermodynamics.NoSolubility object at 0x12776a150>, solid_melt_distribution_coefficient=0, elements={'H': 2}, element_masses={'H': 0.0020158}, formation_constants=(0, 0), molar_mass=0.0020158),
 Molecule(name='O2', solubility=<atmodeller.thermodynamics.NoSolubility object at 0x111e4eb10>, solid_melt_distribution_coefficient=0, elements={'O': 2}, element_masses={'O': 0.0319988}, formation_constants=(0, 0), molar_mass=0.0319988),
 Molecule(name='CO', solubility=<atmodeller.thermodynamics.NoSolubility object at 0x12776a090>, solid_melt_distribution_coefficient=0, elements={'C': 1, 'O': 1}, element_masses={'C': 0.0120107, 'O': 0.0159994}, for

Now we define the constraints, and in this case we want to constrain the total mass of C and H in the system that can partition between the various reservoirs.

In [15]:
number_of_earth_oceans: float = 1
# C/H ratio by mass.
ch_ratio: float = 1

mass_H: float = number_of_earth_oceans * OCEAN_MOLES * MolarMasses().H2
mass_C: float = ch_ratio * mass_H

constraints: list[SystemConstraint] = [
    SystemConstraint(species="H", value=mass_H, field="mass"),
    SystemConstraint(species="C", value=mass_C, field="mass"),
]

We need to apply one more constraint, which is usually the oxygen fugacity and this can be done directly as before by specifying `fo2_constraint=True`:

In [16]:
interior_atmosphere: InteriorAtmosphereSystem = InteriorAtmosphereSystem(molecules=molecules)
interior_atmosphere.solve(constraints, fo2_constraint=True)
interior_atmosphere.pressures_dict

[2023-06-04 14:06:00 - atmodeller.core      - 072 - INFO      - __post_init__()] Creating a new planet
[2023-06-04 14:06:00 - atmodeller.core      - 077 - INFO      - __post_init__()] Mantle mass (kg) = 4.208261222595111e+24
[2023-06-04 14:06:00 - atmodeller.core      - 078 - INFO      - __post_init__()] Mantle melt fraction = 1.0
[2023-06-04 14:06:00 - atmodeller.core      - 079 - INFO      - __post_init__()] Core mass fraction = 0.295334691460966
[2023-06-04 14:06:00 - atmodeller.core      - 080 - INFO      - __post_init__()] Planetary radius (m) = 6371000.0
[2023-06-04 14:06:00 - atmodeller.core      - 081 - INFO      - __post_init__()] Planetary mass (kg) = 5.972e+24
[2023-06-04 14:06:00 - atmodeller.core      - 082 - INFO      - __post_init__()] Surface temperature (K) = 2000.000000
[2023-06-04 14:06:00 - atmodeller.core      - 083 - INFO      - __post_init__()] Surface gravity (m/s^2) = 9.819973426224687
[2023-06-04 14:06:00 - atmodeller.core      - 084 - INFO      - __post_init_

{'CO': 62.30206152706083,
 'H2': 0.5545797824844402,
 'O2': 3.9099946023954954e-08,
 'CO2': 9.469248117924007,
 'H2O': 0.3783451241287987}

### 4. Including more species such as CH4

It is straightforward to add more species to the system, although they must have their formation energies and masses already specified in the code.

In [17]:
molecules: list[Molecule] = []
molecules.append(Molecule(name='H2O', solubility=PeridotiteH2O(), solid_melt_distribution_coefficient=0))
molecules.append(Molecule(name='H2', solubility=NoSolubility(), solid_melt_distribution_coefficient=0))
molecules.append(Molecule(name='O2', solubility=NoSolubility(), solid_melt_distribution_coefficient=0))
molecules.append(Molecule(name='CO', solubility=NoSolubility(), solid_melt_distribution_coefficient=0))
molecules.append(Molecule(name='CO2', solubility=BasaltDixonCO2(), solid_melt_distribution_coefficient=0))
molecules.append(Molecule(name='CH4', solubility=NoSolubility(), solid_melt_distribution_coefficient=0))
molecules

[2023-06-04 14:06:00 - atmodeller.core      - 144 - INFO      - __post_init__()] Creating a molecule: H2O
[2023-06-04 14:06:00 - atmodeller.core      - 144 - INFO      - __post_init__()] Creating a molecule: H2
[2023-06-04 14:06:00 - atmodeller.core      - 144 - INFO      - __post_init__()] Creating a molecule: O2
[2023-06-04 14:06:00 - atmodeller.core      - 144 - INFO      - __post_init__()] Creating a molecule: CO
[2023-06-04 14:06:00 - atmodeller.core      - 144 - INFO      - __post_init__()] Creating a molecule: CO2
[2023-06-04 14:06:00 - atmodeller.core      - 144 - INFO      - __post_init__()] Creating a molecule: CH4


[Molecule(name='H2O', solubility=<atmodeller.thermodynamics.PeridotiteH2O object at 0x12782a110>, solid_melt_distribution_coefficient=0, elements={'H': 2, 'O': 1}, element_masses={'H': 0.0020158, 'O': 0.0159994}, formation_constants=(-3.0385132352941198, 13152.698529411768), molar_mass=0.018015200000000002),
 Molecule(name='H2', solubility=<atmodeller.thermodynamics.NoSolubility object at 0x127764590>, solid_melt_distribution_coefficient=0, elements={'H': 2}, element_masses={'H': 0.0020158}, formation_constants=(0, 0), molar_mass=0.0020158),
 Molecule(name='O2', solubility=<atmodeller.thermodynamics.NoSolubility object at 0x1278426d0>, solid_melt_distribution_coefficient=0, elements={'O': 2}, element_masses={'O': 0.0319988}, formation_constants=(0, 0), molar_mass=0.0319988),
 Molecule(name='CO', solubility=<atmodeller.thermodynamics.NoSolubility object at 0x112502b50>, solid_melt_distribution_coefficient=0, elements={'C': 1, 'O': 1}, element_masses={'C': 0.0120107, 'O': 0.0159994}, for

We define a mixture of mass and oxygen fugacity constraints as before and solve the system. CH4 is not prevalent at 2000 K so the results are almost identical to those without CH4 presented above.

In [18]:
number_of_earth_oceans: float = 1
# C/H ratio by mass.
ch_ratio: float = 1

mass_H: float = number_of_earth_oceans * OCEAN_MOLES * MolarMasses().H2
mass_C: float = ch_ratio * mass_H

constraints: list[SystemConstraint] = [
    SystemConstraint(species="H", value=mass_H, field="mass"),
    SystemConstraint(species="C", value=mass_C, field="mass"),
]

interior_atmosphere: InteriorAtmosphereSystem = InteriorAtmosphereSystem(molecules=molecules)
interior_atmosphere.solve(constraints, fo2_constraint=True)
interior_atmosphere.pressures_dict

[2023-06-04 14:06:00 - atmodeller.core      - 072 - INFO      - __post_init__()] Creating a new planet
[2023-06-04 14:06:00 - atmodeller.core      - 077 - INFO      - __post_init__()] Mantle mass (kg) = 4.208261222595111e+24
[2023-06-04 14:06:00 - atmodeller.core      - 078 - INFO      - __post_init__()] Mantle melt fraction = 1.0
[2023-06-04 14:06:00 - atmodeller.core      - 079 - INFO      - __post_init__()] Core mass fraction = 0.295334691460966
[2023-06-04 14:06:00 - atmodeller.core      - 080 - INFO      - __post_init__()] Planetary radius (m) = 6371000.0
[2023-06-04 14:06:00 - atmodeller.core      - 081 - INFO      - __post_init__()] Planetary mass (kg) = 5.972e+24
[2023-06-04 14:06:00 - atmodeller.core      - 082 - INFO      - __post_init__()] Surface temperature (K) = 2000.000000
[2023-06-04 14:06:00 - atmodeller.core      - 083 - INFO      - __post_init__()] Surface gravity (m/s^2) = 9.819973426224687
[2023-06-04 14:06:00 - atmodeller.core      - 084 - INFO      - __post_init_

{'CO': 62.30205995149259,
 'H2': 0.5545797760445784,
 'O2': 3.9099946023954954e-08,
 'CO2': 9.469247878454462,
 'H2O': 0.3783451197353995,
 'CH4': 1.2820041833656648e-06}